<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/Ensemble_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Print the correct tensorflow version 

In [1]:
from google.colab import drive
try:
  drive.mount('/content/drive/',force_remount = True)
  COLAB = True
  print("Google colab is on")
  %tensorflow_version 2.x
except:
  COLAB = False
  print("Not using Google Colab")

Mounted at /content/drive/
Google colab is on


In [135]:
#Time formatter
def time_fmt(secs):
  hours = int(secs/(60*60))
  minutes = int(secs%(60*60)/60)
  seconds = int(secs%60)
  return f"{hours}:{minutes:>02}:{seconds:>05.2f}"

In [3]:
#Pertubation implementation to variable importance for NNT

In [136]:
import scipy as sc
import numpy as np
from sklearn import metrics
import math
import pandas as pd


In [137]:
#This function will compute the loss for a regression and classification model
#To provide usefull information in computing variable's importance
#First we select all columns from a test dataset
#Second we randomly shuffles every columns to distort any parttern
#If the fitted model is a regression we compute MSE otherwise we compute log-loss
#We compute variable importance by take the ratio of individual error vs maximum error
#We prepare a results dictionary which is finaly conveted to a dataframe.

In [161]:
def var_rank(model, x, y, names, regression):
  errors = []
  for i in range(x.shape[1]):
    hold = np.array(x[:,i])
    np.random.shuffle(x[:,i])
    if regression:
      pred = model.predict(x)
      err = metrics.mean_squared_error(y,pred)
    else:
      pred = model.predict_proba(x)
      err = metrics.log_loss(y, pred)
    errors.append(err)
    x[:,i] = hold
  e_max = np.max(errors) 
  importance = [e/e_max for e in errors]
  data = {'names': names,'errors': errors, 'importance': importance}
  out = pd.DataFrame(data, columns = ['names', 'errors', 'importance'])
  out.sort_values(by = ['importance'], ascending = [0],inplace = True)
  out.reset_index(inplace = True, drop = True)
  return out




In [139]:
#Build a Keras Sequential model for classification using Iris dataset

In [140]:
#Loading the packages
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from sklearn import metrics
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import pandas as pd


In [141]:
iris = pd.read_csv("https://data.heatonresearch.com/data/t81-558/iris.csv",
                   na_values = ['NA','?'])

In [142]:
display(iris.head(10))

,sepal_l,sepal_w,petal_l,petal_w,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [143]:
iris.isna().value_counts()

sepal_l  sepal_w  petal_l  petal_w  species
False    False    False    False    False      150
dtype: int64

In [108]:
dummies = pd.get_dummies(iris['species'])
y = dummies.values



In [144]:
x = iris.drop(columns = ['species'],axis = 1)

In [145]:
x.head()

,sepal_l,sepal_w,petal_l,petal_w
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [146]:
x = x.values
x[0:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [147]:
#Split the data into train and test sets

In [148]:
x_train, x_test, y_train,y_test = train_test_split(x,y, test_size = 0.25, random_state = 42)

In [149]:
x_train[0:2]

array([[5. , 3.6, 1.4, 0.2],
       [5.2, 4.1, 1.5, 0.1]])

In [150]:
y_train[0:2]

array([[1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [151]:
#Building keras Sequential NNT for classifying Iris flowers
model = Sequential()
model.add(Dense(50, input_dim = x.shape[1], activation = 'relu',kernel_initializer = 'random_normal'))
model.add(Dense(25, activation = 'relu', kernel_initializer = 'random_normal'))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
#info = EarlyStopping(monitor = 'val_loss',min_delta = 1e-3,patience = 5,verbose = 1, restore_best_weights = True)
model.fit(x_train, y_train, validation_data = (x_test, y_test), verbose = 2, epochs = 100)

Epoch 1/100
4/4 - 0s - loss: 1.1338 - val_loss: 1.1061
Epoch 2/100
4/4 - 0s - loss: 1.0979 - val_loss: 1.0848
Epoch 3/100
4/4 - 0s - loss: 1.0865 - val_loss: 1.0755
Epoch 4/100
4/4 - 0s - loss: 1.0782 - val_loss: 1.0684
Epoch 5/100
4/4 - 0s - loss: 1.0708 - val_loss: 1.0595
Epoch 6/100
4/4 - 0s - loss: 1.0641 - val_loss: 1.0509
Epoch 7/100
4/4 - 0s - loss: 1.0572 - val_loss: 1.0416
Epoch 8/100
4/4 - 0s - loss: 1.0506 - val_loss: 1.0333
Epoch 9/100
4/4 - 0s - loss: 1.0419 - val_loss: 1.0220
Epoch 10/100
4/4 - 0s - loss: 1.0334 - val_loss: 1.0108
Epoch 11/100
4/4 - 0s - loss: 1.0240 - val_loss: 0.9985
Epoch 12/100
4/4 - 0s - loss: 1.0130 - val_loss: 0.9850
Epoch 13/100
4/4 - 0s - loss: 1.0027 - val_loss: 0.9699
Epoch 14/100
4/4 - 0s - loss: 0.9895 - val_loss: 0.9553
Epoch 15/100
4/4 - 0s - loss: 0.9759 - val_loss: 0.9386
Epoch 16/100
4/4 - 0s - loss: 0.9630 - val_loss: 0.9198
Epoch 17/100
4/4 - 0s - loss: 0.9479 - val_loss: 0.9021
Epoch 18/100
4/4 - 0s - loss: 0.9324 - val_loss: 0.8830
E

In [152]:
preds = model.predict(x_test)

In [153]:
preds = np.argmax(preds, axis = 1)

In [154]:
expected = np.argmax(y_test, axis = 1)

In [155]:
accuracy_score = metrics.accuracy_score(preds, expected)
accuracy_score

0.9736842105263158

In [156]:
#Get the variable importance from the above `NNT
names = list(iris.columns)

In [157]:
 names.remove('species')

In [162]:
from IPython.display import display, HTML


VarImp = var_rank(model, x_test,y_test,names, False)

In [163]:
VarImp

,names,errors,importance
0,petal_l,2.358871,1.000000
1,petal_w,0.829603,0.351695
2,sepal_l,0.156032,0.066147
3,sepal_w,0.119048,0.050468
